[View in Colaboratory](https://colab.research.google.com/github/nicewook/datascience_exercise/blob/master/lane2_Copy1.ipynb)

# Lane 2 분석
## 통계 시작 : 15:00:04 (검정색 말리부 1872) / (CCTV 10:03초 부근)
- 앞 차량 : 검정 소나타
- 뒤 차량 : 검정 SUV
## 통계 종료 : 15:21:34 (흰색 투싼 4052) / (CCTV 31:32초 부근)
- 앞 차량 : 파란색 1톤 포터 (다수의 박스가 화물칸에 있음)
- 뒤 차량 : 흰색 K5


In [0]:
import pandas as pd

# 데이터 읽어오기
## read_csv 
- 경로 및 파일명에 한글이 있으면 오류가 난다. engine='python' 옵션을 주면 된다
- csv 파일에 header 가 없으면 header=None

## column rename
- 방법은 아래 코드 참고

In [0]:
data = pd.read_csv('TriggerLog_1398_2차로/lane2.csv', header=None, engine='python')
data.columns = ["logData"]
data.head()

,logData
0,02000036E920180523145001439010200
1,02000036E920180523145001625020200
2,02000036EA20180523145006841010200
3,02000036EA20180523145007058020200
4,02000036EB20180523145008368010200


## log정보에 millisecond 가 2자리여야 하는데 3자리로 잘못되어 있었음. 
- 그래서 수정함

In [0]:
print(data.dtypes)
print(len(data['logData'][0]))

logData    object
dtype: object
33


In [0]:
data['logData']=data['logData'].str[:26] + data['logData'].str[27:]
print(len(data['logData'][0]))
data.head()

32


,logData
0,02000036E92018052314500143010200
1,02000036E92018052314500162020200
2,02000036EA2018052314500684010200
3,02000036EA2018052314500705020200
4,02000036EB2018052314500836010200


## Time, inOut 정보를 별도 column 으로 만듦
- 분, 초, 밀리초 부분 여섯자리만 가져와서 숫자로 바꿔줌
- inOut 정보 01:in, 02:out

In [0]:
data["dateTime"] = data['logData'].str[20:26]
data["inOut"] = data['logData'].str[26:28]
data.head()

,logData,dateTime,inOut
0,02000036E92018052314500143010200,500143,01
1,02000036E92018052314500162020200,500162,02
2,02000036EA2018052314500684010200,500684,01
3,02000036EA2018052314500705020200,500705,02
4,02000036EB2018052314500836010200,500836,01


## 맨 마지막 행에 값이 없으므로 삭제, dateTime object 를 int32로 변경
- data가 한 줄 줄었다.
- object 를 int32로 변경

In [0]:
print(len(data))
data = data[data.dateTime!=''] 
print(len(data))
data['dateTime']= data['dateTime'].astype('int32')


1749
1748


# 시작시간과 종료시간을 고려해서 불필요한 부분을 날려버림
- 통계 시작 : 15:00:04 (검정색 말리부 1872) / (CCTV 10:03초 부근)
- 통계 종료 : 15:21:34 (흰색 투싼 4052) / (CCTV 31:32초 부근)

In [0]:
data= data[data.dateTime < 213500]
print(len(data))
data.head(10)

994


,logData,dateTime,inOut
404,02000037B32018052315000054010200,54,01
405,02000037B32018052315000075020200,75,02
406,02000037B42018052315000196010200,196,01
407,02000037B42018052315000218020200,218,02
408,02000037B52018052315000423010200,423,01
409,02000037B52018052315000441020200,441,02
410,02000037B62018052315000628010200,628,01
411,02000037B62018052315000643020200,643,02
412,02000037B72018052315000764010200,764,01
413,02000037B72018052315000783020200,783,02


## 중간저장. lane2.csv -> lane2_1.csv

In [0]:
data.to_csv('lane2_1.csv', header=None)

## 진입, 진출을 한 행으로 모아줌

In [0]:
data.head()

,logData,dateTime,inOut
404,02000037B32018052315000054010200,54,01
405,02000037B32018052315000075020200,75,02
406,02000037B42018052315000196010200,196,01
407,02000037B42018052315000218020200,218,02
408,02000037B52018052315000423010200,423,01


In [0]:
print(data.index)
data.reset_index(drop=True, inplace=True)
print(data.index)

Int64Index([ 404,  405,  406,  407,  408,  409,  410,  411,  412,  413,
            ...
            1388, 1389, 1390, 1391, 1392, 1393, 1394, 1395, 1396, 1397],
           dtype='int64', length=994)
RangeIndex(start=0, stop=994, step=1)


In [0]:
print(data.head())
print(data.tail())

                            logData  dateTime inOut
0  02000037B32018052315000054010200        54    01
1  02000037B32018052315000075020200        75    02
2  02000037B42018052315000196010200       196    01
3  02000037B42018052315000218020200       218    02
4  02000037B52018052315000423010200       423    01
                              logData  dateTime inOut
989  02000039A12018052315212780020200    212780    02
990  02000039A22018052315213016010200    213016    01
991  02000039A22018052315213044020200    213044    02
992  02000039A32018052315213339010200    213339    01
993  02000039A32018052315213355020200    213355    02


In [0]:
data1 = data.iloc[::2]
data1.reset_index(drop=True, inplace=True)
print(len(data1))
data2 = data.iloc[1::2]
data2.reset_index(drop=True, inplace=True)
print(len(data2))
data2.head()

497
497


,logData,dateTime,inOut
0,02000037B32018052315000075020200,75,02
1,02000037B42018052315000218020200,218,02
2,02000037B52018052315000441020200,441,02
3,02000037B62018052315000643020200,643,02
4,02000037B72018052315000783020200,783,02


In [0]:
#data1[['logData2', 'dateTime2', 'inOut2']] = [data2['logData'], data2['dateTime'], data2['inOut']]
data3 = pd.concat([data1, data2], axis=1)
data3.head()

,logData,dateTime,inOut,logData,dateTime,inOut
0,02000037B32018052315000054010200,54,01,02000037B32018052315000075020200,75,02
1,02000037B42018052315000196010200,196,01,02000037B42018052315000218020200,218,02
2,02000037B52018052315000423010200,423,01,02000037B52018052315000441020200,441,02
3,02000037B62018052315000628010200,628,01,02000037B62018052315000643020200,643,02
4,02000037B72018052315000764010200,764,01,02000037B72018052315000783020200,783,02


In [0]:
print(data3.columns)
len(data3)

Index(['logData', 'dateTime', 'inOut', 'logData', 'dateTime', 'inOut'], dtype='object')


497

In [0]:
data3.columns =['logData', 'dateTime', 'inOut', 'logData2', 'dateTime2', 'inOut2']
data3.columns

Index(['logData', 'dateTime', 'inOut', 'logData2', 'dateTime2', 'inOut2'], dtype='object')

In [0]:
data3.head()

,logData,dateTime,inOut,logData2,dateTime2,inOut2
0,02000037B32018052315000054010200,54,01,02000037B32018052315000075020200,75,02
1,02000037B42018052315000196010200,196,01,02000037B42018052315000218020200,218,02
2,02000037B52018052315000423010200,423,01,02000037B52018052315000441020200,441,02
3,02000037B62018052315000628010200,628,01,02000037B62018052315000643020200,643,02
4,02000037B72018052315000764010200,764,01,02000037B72018052315000783020200,783,02


In [0]:
data3.to_csv('lane2_2.csv', header=None)

## .srt 자막파일을 만들어보자
- .srt 포맷: https://www.3playmedia.com/2017/03/08/create-srt-file/

In [0]:
data = data3.copy()

In [0]:
data.dateTime = data.dateTime +100300 - 54  # 영상의 10:03초 부터 시작됨, 차량 인식의 시작이 54 부터임
data.dateTime2 = data.dateTime2 +100300 - 54  # 영상의 10:03초 부터 시작됨, 차량 인식의 시작이 54 부터임
data.head()

,logData,dateTime,inOut,logData2,dateTime2,inOut2
0,02000037B32018052315000054010200,100300,01,02000037B32018052315000075020200,100321,02
1,02000037B42018052315000196010200,100442,01,02000037B42018052315000218020200,100464,02
2,02000037B52018052315000423010200,100669,01,02000037B52018052315000441020200,100687,02
3,02000037B62018052315000628010200,100874,01,02000037B62018052315000643020200,100889,02
4,02000037B72018052315000764010200,101010,01,02000037B72018052315000783020200,101029,02


In [0]:
data.dateTime = data.dateTime.astype('str')
data.dateTime2 = data.dateTime2.astype('str')

# make start string
data['start'] = "00:" + data.dateTime.str[:2] \
            + ":" + data.dateTime.str[2:4] \
            + "," + data.dateTime.str[4:] \
            + "0 --> "
# make stop string
data['start'] = data['start'].str[:] \
            + "00:" + data.dateTime2.str[:2] \
            + ":" + data.dateTime2.str[2:4] \
            + "," + data.dateTime2.str[4:] \
            + "0"
print(data.head())

                            logData dateTime inOut  \
0  02000037B32018052315000054010200   100300    01   
1  02000037B42018052315000196010200   100442    01   
2  02000037B52018052315000423010200   100669    01   
3  02000037B62018052315000628010200   100874    01   
4  02000037B72018052315000764010200   101010    01   

                           logData2 dateTime2 inOut2  \
0  02000037B32018052315000075020200    100321     02   
1  02000037B42018052315000218020200    100464     02   
2  02000037B52018052315000441020200    100687     02   
3  02000037B62018052315000643020200    100889     02   
4  02000037B72018052315000783020200    101029     02   

                           start  
0  00:10:03,000 --> 00:10:03,210  
1  00:10:04,420 --> 00:10:04,640  
2  00:10:06,690 --> 00:10:06,870  
3  00:10:08,740 --> 00:10:08,890  
4  00:10:10,100 --> 00:10:10,290  


In [0]:
data['subtitle'] = data.index.astype(int) + 1
data.head()

,logData,dateTime,inOut,logData2,dateTime2,inOut2,start,subtitle
0,02000037B32018052315000054010200,100300,01,02000037B32018052315000075020200,100321,02,"00:10:03,000 --> 00:10:03,210",1
1,02000037B42018052315000196010200,100442,01,02000037B42018052315000218020200,100464,02,"00:10:04,420 --> 00:10:04,640",2
2,02000037B52018052315000423010200,100669,01,02000037B52018052315000441020200,100687,02,"00:10:06,690 --> 00:10:06,870",3
3,02000037B62018052315000628010200,100874,01,02000037B62018052315000643020200,100889,02,"00:10:08,740 --> 00:10:08,890",4
4,02000037B72018052315000764010200,101010,01,02000037B72018052315000783020200,101029,02,"00:10:10,100 --> 00:10:10,290",5


In [0]:
srtdata = data[['start', 'subtitle']].copy()
print(srtdata.dtypes)

srtdata.subtitle = srtdata.subtitle.astype('str')

print(srtdata.dtypes)
srtdata.head()

start       object
subtitle     int64
dtype: object
start       object
subtitle    object
dtype: object


,start,subtitle
0,"00:10:03,000 --> 00:10:03,210",1
1,"00:10:04,420 --> 00:10:04,640",2
2,"00:10:06,690 --> 00:10:06,870",3
3,"00:10:08,740 --> 00:10:08,890",4
4,"00:10:10,100 --> 00:10:10,290",5


In [0]:
srtdata['final'] = srtdata['subtitle'].str[:] + "\n" \
                    + srtdata['start'].str[:] + "\n" \
                    + "car count: " \
                    + srtdata['subtitle'].str[:] + "\n\n"
#srtdata = srtdata['final']
srtdata.head()

,start,subtitle,final
0,"00:10:03,000 --> 00:10:03,210",1,"1\n00:10:03,000 --> 00:10:03,210\ncar count: 1..."
1,"00:10:04,420 --> 00:10:04,640",2,"2\n00:10:04,420 --> 00:10:04,640\ncar count: 2..."
2,"00:10:06,690 --> 00:10:06,870",3,"3\n00:10:06,690 --> 00:10:06,870\ncar count: 3..."
3,"00:10:08,740 --> 00:10:08,890",4,"4\n00:10:08,740 --> 00:10:08,890\ncar count: 4..."
4,"00:10:10,100 --> 00:10:10,290",5,"5\n00:10:10,100 --> 00:10:10,290\ncar count: 5..."


In [0]:
srtdata = srtdata.final

In [0]:
srtdata.head()

0    1\n00:10:03,000 --> 00:10:03,210\ncar count: 1...
1    2\n00:10:04,420 --> 00:10:04,640\ncar count: 2...
2    3\n00:10:06,690 --> 00:10:06,870\ncar count: 3...
3    4\n00:10:08,740 --> 00:10:08,890\ncar count: 4...
4    5\n00:10:10,100 --> 00:10:10,290\ncar count: 5...
Name: final, dtype: object

In [0]:
srtdata.to_csv?

Signature: srtdata.to_csv(path=None, index=True, sep=',', na_rep='', float_format=None, header=False, index_label=None, mode='w', encoding=None, date_format=None, decimal='.')
Docstring:
Write Series to a comma-separated values (csv) file

Parameters
----------
path : string or file handle, default None
    File path or object, if None is provided the result is returned as
    a string.
na_rep : string, default ''
    Missing data representation
float_format : string, default None
    Format string for floating point numbers
header : boolean, default False
    Write out series name
index : boolean, default True
    Write row names (index)
index_label : string or sequence, default None
    Column label for index column(s) if desired. If None is given, and
    `header` and `index` are True, then the index names are used. A
    sequence should be given if the DataFrame uses MultiIndex.
mode : Python write mode, default 'w'
sep : character, default ","
    Field delimiter for the output fi

In [0]:
#srtdata.to_csv('lane2.srt', header=None, index=False).drop(['unnamed 0'],axis=1)
srtdata.to_csv('lane2.srt', header=None, mode = 'w', index=False)
#srtdata = pd.read_csv('temp.csv', header=None, index_col=False)
#srtdata.to_csv('lane2.srt', header=None, index=False)

In [0]:
print(len(srtdata))